In [338]:
import pandas as pd
import numpy as np

from collections import Counter


from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")


In [339]:
train = pd.read_csv(r"C:\Users\91939\Documents\DA\ML\titanic\train.csv")
test = pd.read_csv(r"C:\Users\91939\Documents\DA\ML\titanic\test.csv")
ss = pd.read_csv(r"C:\Users\91939\Documents\DA\ML\titanic\gender_submission.csv")

In [340]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [341]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [342]:
ss.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [343]:
print("Training set shape", train.shape)
print("Testing set shape", test.shape)
print("Submission set shape", ss.shape)

Training set shape (891, 12)
Testing set shape (418, 11)
Submission set shape (418, 2)


In [344]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [345]:
train.isnull().sum().sort_values(ascending=False)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

In [346]:
test.isnull().sum().sort_values(ascending=False)

Cabin          327
Age             86
Fare             1
PassengerId      0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Embarked         0
dtype: int64

In [347]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [348]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [349]:
train['Sex'].value_counts(dropna=False)

male      577
female    314
Name: Sex, dtype: int64

In [350]:
train[['Sex','Survived']].groupby('Sex',as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [351]:
train['Pclass'].value_counts(dropna=False)

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [352]:
train[['Pclass','Survived']].groupby('Pclass',as_index=False).mean().sort_values(by='Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [353]:
train['Embarked'].value_counts(dropna=False)

S      644
C      168
Q       77
NaN      2
Name: Embarked, dtype: int64

In [354]:
train[['Embarked','Survived']].groupby('Embarked',as_index=False).mean().sort_values(by='Survived',ascending=False)

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.336957


In [355]:
def detect_outliers(df,n,features):
    outlier_indices=[]
    for col in features:
        Q1=np.percentile(df[col],25)
        Q3=np.percentile(df[col],75)
        IQR=Q3-Q1
        outlier_step=1.5*IQR
        outlier_list_col=df[(df[col]<Q1-outlier_step)|(df[col]>Q3+outlier_step)].index
        outlier_indices.extend(outlier_list_col)
    outlier_indices=Counter(outlier_indices)
    multiple_outliers=list(key for key, value in outlier_indices.items() if value>n)
    return multiple_outliers

outliers_to_drop= detect_outliers(train,2,['Age','SibSp','Parch','Fare'])
print("We will drop these {} indices:".format(len(outliers_to_drop)), outliers_to_drop)


We will drop these 10 indices: [27, 88, 159, 180, 201, 324, 341, 792, 846, 863]


In [356]:
train.loc[outliers_to_drop,:]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.00,C23 C25 C27,S
88,89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.00,C23 C25 C27,S
159,160,0,3,"Sage, Master. Thomas Henry",male,NaN,8,2,CA. 2343,69.55,NaN,S
180,181,0,3,"Sage, Miss. Constance Gladys",female,NaN,8,2,CA. 2343,69.55,NaN,S
201,202,0,3,"Sage, Mr. Frederick",male,NaN,8,2,CA. 2343,69.55,NaN,S
324,325,0,3,"Sage, Mr. George John Jr",male,NaN,8,2,CA. 2343,69.55,NaN,S
341,342,1,1,"Fortune, Miss. Alice Elizabeth",female,24.0,3,2,19950,263.00,C23 C25 C27,S
792,793,0,3,"Sage, Miss. Stella Anna",female,NaN,8,2,CA. 2343,69.55,NaN,S
846,847,0,3,"Sage, Mr. Douglas Bullen",male,NaN,8,2,CA. 2343,69.55,NaN,S
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.55,NaN,S


In [357]:
print("Before: {}".format(len(train)))
train=train.drop(outliers_to_drop,axis=0).reset_index(drop=True)
print("After: {}".format(len(train)))

Before: 891
After: 881


In [358]:
train['SibSp'].value_counts(dropna=False)

0    608
1    209
2     28
4     18
3     13
5      5
Name: SibSp, dtype: int64

In [359]:
train[['SibSp','Survived']].groupby('SibSp',as_index=False).mean().sort_values(by='Survived', ascending=False)

,SibSp,Survived
1,1,0.535885
2,2,0.464286
0,0,0.345395
4,4,0.166667
3,3,0.153846
5,5,0.000000


In [360]:
train['Parch'].value_counts(dropna=False)

0    678
1    118
2     70
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

In [361]:
train[['Parch','Survived']].groupby('Parch',as_index=False).mean().sort_values(by='Survived', ascending=False)

,Parch,Survived
3,3,0.600000
1,1,0.550847
2,2,0.542857
0,0,0.343658
5,5,0.200000
4,4,0.000000
6,6,0.000000


In [362]:
train['Age'].isnull().sum()

170

In [363]:
train['Fare'].isnull().sum()

0

In [364]:
train=train.drop(['Ticket','Cabin'],axis=1)
test=test.drop(['Ticket','Cabin'],axis=1)

In [365]:
train.isnull().sum().sort_values(ascending=False)

Age            170
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Fare             0
dtype: int64

In [366]:
mode=train['Embarked'].dropna().mode()[0]
mode

'S'

In [367]:
train['Embarked'].fillna(mode,inplace=True)

In [368]:
test.isnull().sum().sort_values(ascending=False)

Age            86
Fare            1
PassengerId     0
Pclass          0
Name            0
Sex             0
SibSp           0
Parch           0
Embarked        0
dtype: int64

In [369]:
median=test['Fare'].dropna().median()
median

14.4542

In [370]:
test['Fare'].fillna(median,inplace=True)

In [371]:
combine=pd.concat([train,test],axis=0).reset_index(drop=True)
combine.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [372]:
combine.isnull().sum().sort_values(ascending=False)

Survived       418
Age            256
PassengerId      0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Fare             0
Embarked         0
dtype: int64

In [373]:
combine['Sex']=combine['Sex'].map({'male':0,'female':1})

In [374]:
age_nan_indices=list(combine[combine['Age'].isnull()].index)
len(age_nan_indices)

256

In [375]:
for index in age_nan_indices:
    median_age=combine['Age'].median()
    predict_age=combine['Age'][(combine['SibSp']==combine.iloc[index]['SibSp']) & 
                              (combine['Parch']==combine.iloc[index]['Parch']) & 
                              (combine['Pclass']==combine.iloc[index]['Pclass'])].median()
    if np.isnan(predict_age):
        combine['Age'].iloc[index]=median_age
    else:
        combine['Age'].iloc[index]=predict_age

In [376]:
combine['Age'].isnull().sum()

0

In [377]:
combine['Fare']=combine['Fare'].map(lambda x:np.log(x) if x>0 else 0)

In [378]:
combine.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,1.981001,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,4.266662,C
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,2.070022,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,3.972177,S
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,2.085672,S


In [379]:
combine['Title']=[name.split(',')[1].split('.')[0].strip() for name in combine['Name']]
combine[['Name','Title']].head()

,Name,Title
0,"Braund, Mr. Owen Harris",Mr
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Mrs
2,"Heikkinen, Miss. Laina",Miss
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Mrs
4,"Allen, Mr. William Henry",Mr


In [380]:
combine['Title'].value_counts(ascending= False)

Mr              753
Miss            255
Mrs             197
Master           60
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Lady              1
Sir               1
Mme               1
Don               1
Capt              1
the Countess      1
Jonkheer          1
Dona              1
Name: Title, dtype: int64

In [381]:
combine['Title'].nunique()

18

In [382]:
combine['Title']=combine['Title'].replace(['Dr','Rev','Col','Major','Lady','Jonkheer','Don','Capt','the Countess','Sir','Dona'],
                                         'Rare')
combine['Title']=combine['Title'].replace(['Mlle','Ms'],'Miss')
combine['Title']=combine['Title'].replace('Mme','Mrs')

In [383]:
combine[['Title','Survived']].groupby('Title',as_index=False).mean().sort_values(by='Survived', ascending=False)

,Title,Survived
3,Mrs,0.793651
1,Miss,0.711111
0,Master,0.589744
4,Rare,0.347826
2,Mr,0.157895


In [384]:
combine=combine.drop('Name',axis=1)
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0.0,3,0,22.0,1,0,1.981001,S,Mr
1,2,1.0,1,1,38.0,1,0,4.266662,C,Mrs
2,3,1.0,3,1,26.0,0,0,2.070022,S,Miss
3,4,1.0,1,1,35.0,1,0,3.972177,S,Mrs
4,5,0.0,3,0,35.0,0,0,2.085672,S,Mr


In [385]:
combine['Family Size']=combine['SibSp']+combine['Parch']+1
combine[['SibSp','Parch','Family Size']].head(10)

,SibSp,Parch,Family Size
0,1,0,2
1,1,0,2
2,0,0,1
3,1,0,2
4,0,0,1
5,0,0,1
6,0,0,1
7,3,1,5
8,0,2,3
9,1,0,2


In [386]:
combine[['Family Size','Survived']].groupby('Family Size',as_index=False).mean().sort_values(by='Survived', ascending=False)

,Family Size,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.052632
7,8,0.000000
8,11,NaN


In [387]:
combine['IsAlone']=0
combine.loc[combine['Family Size']==1,'IsAlone']=1

In [388]:
combine[['IsAlone','Survived']].groupby('IsAlone',as_index=False).mean().sort_values(by='IsAlone', ascending=False)

,IsAlone,Survived
1,1,0.303538
0,0,0.514535


In [389]:
combine=combine.drop(['SibSp','Parch','Family Size'],axis=1)
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,1,0.0,3,0,22.0,1.981001,S,Mr,0
1,2,1.0,1,1,38.0,4.266662,C,Mrs,0
2,3,1.0,3,1,26.0,2.070022,S,Miss,1
3,4,1.0,1,1,35.0,3.972177,S,Mrs,0
4,5,0.0,3,0,35.0,2.085672,S,Mr,1


In [390]:
combine['AgeBand']=pd.cut(combine['Age'],5)
combine[['AgeBand','Survived']].groupby('AgeBand',as_index=False).mean().sort_values(by='AgeBand', ascending=False)

,AgeBand,Survived
4,"(64.034, 80.0]",0.090909
3,"(48.068, 64.034]",0.434783
2,"(32.102, 48.068]",0.412844
1,"(16.136, 32.102]",0.339662
0,"(0.0902, 16.136]",0.532110


In [391]:
combine.loc[combine['Age']<=16.136,'Age'] = 0
combine.loc[(combine['Age']>16.136) & (combine['Age']<=32.102),'Age']=1
combine.loc[(combine['Age']>32.102) & (combine['Age']<=48.068),'Age']=2
combine.loc[(combine['Age']>48.068) & (combine['Age']<=64.034),'Age']=3
combine.loc[combine['Age']>64.034,'Age']=4

In [392]:
combine=combine.drop('AgeBand',axis=1)

In [393]:
combine[['Age','Pclass']].dtypes

Age       float64
Pclass      int64
dtype: object

In [394]:
combine['Age']=combine['Age'].astype('int')
combine['Age'].dtypes

dtype('int32')

In [395]:
combine['Age*Class']=combine['Age']*combine['Pclass']
combine[['Age','Pclass','Age*Class']].head()


,Age,Pclass,Age*Class
0,1,3,3
1,2,1,2
2,1,3,3
3,2,1,2
4,2,3,6


In [396]:
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,1,0.0,3,0,1,1.981001,S,Mr,0,3
1,2,1.0,1,1,2,4.266662,C,Mrs,0,2
2,3,1.0,3,1,1,2.070022,S,Miss,1,3
3,4,1.0,1,1,2,3.972177,S,Mrs,0,2
4,5,0.0,3,0,2,2.085672,S,Mr,1,6


In [397]:
combine=pd.get_dummies(combine,columns=['Title'])
combine=pd.get_dummies(combine,columns=['Embarked'], prefix='Em')
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
0,1,0.0,3,0,1,1.981001,0,3,0,0,1,0,0,0,0,1
1,2,1.0,1,1,2,4.266662,0,2,0,0,0,1,0,1,0,0
2,3,1.0,3,1,1,2.070022,1,3,0,1,0,0,0,0,0,1
3,4,1.0,1,1,2,3.972177,0,2,0,0,0,1,0,0,0,1
4,5,0.0,3,0,2,2.085672,1,6,0,0,1,0,0,0,0,1


In [398]:
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
0,1,0.0,3,0,1,1.981001,0,3,0,0,1,0,0,0,0,1
1,2,1.0,1,1,2,4.266662,0,2,0,0,0,1,0,1,0,0
2,3,1.0,3,1,1,2.070022,1,3,0,1,0,0,0,0,0,1
3,4,1.0,1,1,2,3.972177,0,2,0,0,0,1,0,0,0,1
4,5,0.0,3,0,2,2.085672,1,6,0,0,1,0,0,0,0,1


In [399]:
combine['FareBand']=pd.cut(combine['Fare'],5)
combine[['FareBand','Survived']].groupby('FareBand',as_index=False).mean().sort_values(by='FareBand', ascending=False)

,FareBand,Survived
4,"(4.991, 6.239]",0.692308
3,"(3.743, 4.991]",0.669118
2,"(2.496, 3.743]",0.431884
1,"(1.248, 2.496]",0.225627
0,"(-0.00624, 1.248]",0.066667


In [400]:
combine.loc[combine['Fare']<=1.248,'Fare'] = 0
combine.loc[(combine['Fare']>1.248) & (combine['Fare']<=2.496),'Fare']=1
combine.loc[(combine['Fare']>2.496) & (combine['Fare']<=3.743),'Fare']=2
combine.loc[(combine['Fare']>3.743) & (combine['Fare']<=4.991),'Fare']=3
combine.loc[combine['Fare']>4.991,'Fare']=4

In [401]:
combine=combine.drop('FareBand',axis=1)

In [402]:

combine['Fare'].dtypes

dtype('float64')

In [403]:
combine['Fare']=combine['Fare'].astype('int')
combine['Fare'].dtypes

dtype('int32')

In [404]:
train=combine[:len(train)]
test=combine[len(train):]

In [405]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
0,1,0.0,3,0,1,1,0,3,0,0,1,0,0,0,0,1
1,2,1.0,1,1,2,3,0,2,0,0,0,1,0,1,0,0
2,3,1.0,3,1,1,1,1,3,0,1,0,0,0,0,0,1
3,4,1.0,1,1,2,3,0,2,0,0,0,1,0,0,0,1
4,5,0.0,3,0,2,1,1,6,0,0,1,0,0,0,0,1


In [406]:
train=train.drop('PassengerId',axis=1)
train.head()

,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
0,0.0,3,0,1,1,0,3,0,0,1,0,0,0,0,1
1,1.0,1,1,2,3,0,2,0,0,0,1,0,1,0,0
2,1.0,3,1,1,1,1,3,0,1,0,0,0,0,0,1
3,1.0,1,1,2,3,0,2,0,0,0,1,0,0,0,1
4,0.0,3,0,2,1,1,6,0,0,1,0,0,0,0,1


In [407]:
train['Survived']=train['Survived'].astype('int')
train.head()

,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
0,0,3,0,1,1,0,3,0,0,1,0,0,0,0,1
1,1,1,1,2,3,0,2,0,0,0,1,0,1,0,0
2,1,3,1,1,1,1,3,0,1,0,0,0,0,0,1
3,1,1,1,2,3,0,2,0,0,0,1,0,0,0,1
4,0,3,0,2,1,1,6,0,0,1,0,0,0,0,1


In [408]:
test.head()


,PassengerId,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
881,892,NaN,3,0,2,1,1,6,0,0,1,0,0,0,1,0
882,893,NaN,3,1,2,1,0,6,0,0,0,1,0,0,0,1
883,894,NaN,2,0,3,1,1,6,0,0,1,0,0,0,1,0
884,895,NaN,3,0,1,1,1,3,0,0,1,0,0,0,0,1
885,896,NaN,3,1,1,2,0,3,0,0,0,1,0,0,0,1


In [409]:
test=test.drop('Survived',axis=1)
test.head()

,PassengerId,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
881,892,3,0,2,1,1,6,0,0,1,0,0,0,1,0
882,893,3,1,2,1,0,6,0,0,0,1,0,0,0,1
883,894,2,0,3,1,1,6,0,0,1,0,0,0,1,0
884,895,3,0,1,1,1,3,0,0,1,0,0,0,0,1
885,896,3,1,1,2,0,3,0,0,0,1,0,0,0,1


In [410]:
test.head()

,PassengerId,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Em_C,Em_Q,Em_S
881,892,3,0,2,1,1,6,0,0,1,0,0,0,1,0
882,893,3,1,2,1,0,6,0,0,0,1,0,0,0,1
883,894,2,0,3,1,1,6,0,0,1,0,0,0,1,0
884,895,3,0,1,1,1,3,0,0,1,0,0,0,0,1
885,896,3,1,1,2,0,3,0,0,0,1,0,0,0,1


In [411]:
X_train=train.drop('Survived',axis=1)
Y_train= train['Survived']
X_test=test.drop('PassengerId',axis=1)
print('X_train: ', X_train.shape)
print('Y_train: ', Y_train.shape)
print('X_test: ', X_test.shape)

X_train:  (881, 14)
Y_train:  (881,)
X_test:  (418, 14)


In [412]:
logreg=LogisticRegression()
logreg.fit(X_train,Y_train)
Y_pred=logreg.predict(X_test)
acc_log=round(logreg.score(X_train,Y_train)*100,2)
acc_log

80.36

In [413]:
svc=SVC()
svc.fit(X_train,Y_train)
Y_pred=svc.predict(X_test)
acc_svc=round(svc.score(X_train,Y_train)*100,2)
acc_svc

81.73

In [414]:
gaussian=GaussianNB()
gaussian.fit(X_train,Y_train)
Y_pred=gaussian.predict(X_test)
acc_gaussian=round(gaussian.score(X_train,Y_train)*100,2)
acc_gaussian

79.57

In [415]:
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,Y_train)
Y_pred=knn.predict(X_test)
acc_knn=round(knn.score(X_train,Y_train)*100,2)
acc_knn

83.54

In [416]:
per=Perceptron()
per.fit(X_train,Y_train)
Y_pred=per.predict(X_test)
acc_per=round(per.score(X_train,Y_train)*100,2)
acc_per

76.62

In [417]:
linearsvc=LinearSVC()
linearsvc.fit(X_train,Y_train)
Y_pred=linearsvc.predict(X_test)
acc_linearsvc=round(linearsvc.score(X_train,Y_train)*100,2)
acc_linearsvc

80.48

In [418]:
sgd=SGDClassifier()
sgd.fit(X_train,Y_train)
Y_pred=sgd.predict(X_test)
acc_sgd=round(sgd.score(X_train,Y_train)*100,2)
acc_sgd

80.02

In [419]:
dtc=DecisionTreeClassifier()
dtc.fit(X_train,Y_train)
Y_pred=dtc.predict(X_test)
acc_dtc=round(dtc.score(X_train,Y_train)*100,2)
acc_dtc

86.04

In [420]:
rf=RandomForestClassifier(n_estimators=100)
rf.fit(X_train,Y_train)
Y_pred=rf.predict(X_test)
acc_rf=round(rf.score(X_train,Y_train)*100,2)
acc_rf

86.04

In [421]:
cb=CatBoostClassifier()
cb.fit(X_train,Y_train)
Y_pred=cb.predict(X_test)
acc_cb=round(dtc.score(X_train,Y_train)*100,2)
acc_cb

Learning rate set to 0.00976
0:	learn: 0.6862241	total: 2.38ms	remaining: 2.38s
1:	learn: 0.6800601	total: 4.76ms	remaining: 2.38s
2:	learn: 0.6737396	total: 6.26ms	remaining: 2.08s
3:	learn: 0.6677584	total: 7.6ms	remaining: 1.89s
4:	learn: 0.6615680	total: 8.88ms	remaining: 1.77s
5:	learn: 0.6562306	total: 10.5ms	remaining: 1.74s
6:	learn: 0.6508071	total: 12.6ms	remaining: 1.78s
7:	learn: 0.6451461	total: 14.6ms	remaining: 1.81s
8:	learn: 0.6412020	total: 15.7ms	remaining: 1.73s
9:	learn: 0.6365301	total: 16.9ms	remaining: 1.68s
10:	learn: 0.6319632	total: 17.9ms	remaining: 1.61s
11:	learn: 0.6265906	total: 19ms	remaining: 1.56s
12:	learn: 0.6216745	total: 20ms	remaining: 1.52s
13:	learn: 0.6176746	total: 21ms	remaining: 1.48s
14:	learn: 0.6140204	total: 22.7ms	remaining: 1.49s
15:	learn: 0.6099341	total: 24.6ms	remaining: 1.51s
16:	learn: 0.6054250	total: 25.7ms	remaining: 1.49s
17:	learn: 0.6027931	total: 26.6ms	remaining: 1.45s
18:	learn: 0.5984658	total: 27.6ms	remaining: 1.43s


171:	learn: 0.4000176	total: 189ms	remaining: 911ms
172:	learn: 0.3996036	total: 191ms	remaining: 911ms
173:	learn: 0.3992453	total: 192ms	remaining: 912ms
174:	learn: 0.3989253	total: 193ms	remaining: 911ms
175:	learn: 0.3986240	total: 195ms	remaining: 912ms
176:	learn: 0.3982727	total: 196ms	remaining: 911ms
177:	learn: 0.3981230	total: 197ms	remaining: 910ms
178:	learn: 0.3977622	total: 198ms	remaining: 909ms
179:	learn: 0.3974020	total: 199ms	remaining: 908ms
180:	learn: 0.3971440	total: 201ms	remaining: 908ms
181:	learn: 0.3970061	total: 202ms	remaining: 906ms
182:	learn: 0.3966989	total: 203ms	remaining: 906ms
183:	learn: 0.3962850	total: 204ms	remaining: 906ms
184:	learn: 0.3960116	total: 206ms	remaining: 906ms
185:	learn: 0.3957550	total: 207ms	remaining: 907ms
186:	learn: 0.3955042	total: 209ms	remaining: 909ms
187:	learn: 0.3953279	total: 211ms	remaining: 909ms
188:	learn: 0.3952107	total: 212ms	remaining: 909ms
189:	learn: 0.3950664	total: 213ms	remaining: 907ms
190:	learn: 

341:	learn: 0.3711225	total: 378ms	remaining: 728ms
342:	learn: 0.3710142	total: 380ms	remaining: 727ms
343:	learn: 0.3707850	total: 381ms	remaining: 726ms
344:	learn: 0.3707336	total: 382ms	remaining: 726ms
345:	learn: 0.3705978	total: 384ms	remaining: 725ms
346:	learn: 0.3705882	total: 385ms	remaining: 725ms
347:	learn: 0.3704690	total: 387ms	remaining: 724ms
348:	learn: 0.3703382	total: 388ms	remaining: 724ms
349:	learn: 0.3702459	total: 390ms	remaining: 724ms
350:	learn: 0.3701035	total: 391ms	remaining: 723ms
351:	learn: 0.3700305	total: 392ms	remaining: 723ms
352:	learn: 0.3699518	total: 394ms	remaining: 722ms
353:	learn: 0.3698385	total: 395ms	remaining: 721ms
354:	learn: 0.3697101	total: 397ms	remaining: 720ms
355:	learn: 0.3696154	total: 399ms	remaining: 721ms
356:	learn: 0.3695242	total: 400ms	remaining: 721ms
357:	learn: 0.3694127	total: 402ms	remaining: 720ms
358:	learn: 0.3693514	total: 403ms	remaining: 719ms
359:	learn: 0.3692590	total: 404ms	remaining: 718ms
360:	learn: 

521:	learn: 0.3561375	total: 578ms	remaining: 530ms
522:	learn: 0.3560397	total: 580ms	remaining: 529ms
523:	learn: 0.3559816	total: 581ms	remaining: 528ms
524:	learn: 0.3559254	total: 583ms	remaining: 527ms
525:	learn: 0.3558783	total: 584ms	remaining: 526ms
526:	learn: 0.3557998	total: 585ms	remaining: 525ms
527:	learn: 0.3557515	total: 586ms	remaining: 524ms
528:	learn: 0.3557437	total: 587ms	remaining: 523ms
529:	learn: 0.3557334	total: 588ms	remaining: 522ms
530:	learn: 0.3556818	total: 590ms	remaining: 521ms
531:	learn: 0.3556746	total: 591ms	remaining: 519ms
532:	learn: 0.3555984	total: 592ms	remaining: 519ms
533:	learn: 0.3555497	total: 593ms	remaining: 518ms
534:	learn: 0.3555440	total: 594ms	remaining: 517ms
535:	learn: 0.3554288	total: 595ms	remaining: 515ms
536:	learn: 0.3553679	total: 596ms	remaining: 514ms
537:	learn: 0.3553099	total: 598ms	remaining: 513ms
538:	learn: 0.3552590	total: 599ms	remaining: 512ms
539:	learn: 0.3552580	total: 599ms	remaining: 511ms
540:	learn: 

703:	learn: 0.3467916	total: 770ms	remaining: 324ms
704:	learn: 0.3467487	total: 772ms	remaining: 323ms
705:	learn: 0.3467170	total: 773ms	remaining: 322ms
706:	learn: 0.3466787	total: 774ms	remaining: 321ms
707:	learn: 0.3466407	total: 776ms	remaining: 320ms
708:	learn: 0.3466393	total: 777ms	remaining: 319ms
709:	learn: 0.3466125	total: 778ms	remaining: 318ms
710:	learn: 0.3465567	total: 779ms	remaining: 317ms
711:	learn: 0.3464798	total: 781ms	remaining: 316ms
712:	learn: 0.3464468	total: 782ms	remaining: 315ms
713:	learn: 0.3463972	total: 784ms	remaining: 314ms
714:	learn: 0.3463881	total: 785ms	remaining: 313ms
715:	learn: 0.3463273	total: 786ms	remaining: 312ms
716:	learn: 0.3463163	total: 787ms	remaining: 311ms
717:	learn: 0.3462715	total: 788ms	remaining: 310ms
718:	learn: 0.3462440	total: 789ms	remaining: 308ms
719:	learn: 0.3462094	total: 790ms	remaining: 307ms
720:	learn: 0.3461801	total: 792ms	remaining: 306ms
721:	learn: 0.3461122	total: 793ms	remaining: 305ms
722:	learn: 

888:	learn: 0.3378038	total: 976ms	remaining: 122ms
889:	learn: 0.3377638	total: 977ms	remaining: 121ms
890:	learn: 0.3376786	total: 978ms	remaining: 120ms
891:	learn: 0.3376251	total: 980ms	remaining: 119ms
892:	learn: 0.3375709	total: 981ms	remaining: 118ms
893:	learn: 0.3375310	total: 982ms	remaining: 116ms
894:	learn: 0.3374912	total: 984ms	remaining: 115ms
895:	learn: 0.3374594	total: 985ms	remaining: 114ms
896:	learn: 0.3374119	total: 986ms	remaining: 113ms
897:	learn: 0.3373565	total: 988ms	remaining: 112ms
898:	learn: 0.3373013	total: 989ms	remaining: 111ms
899:	learn: 0.3372202	total: 991ms	remaining: 110ms
900:	learn: 0.3371707	total: 992ms	remaining: 109ms
901:	learn: 0.3371229	total: 993ms	remaining: 108ms
902:	learn: 0.3370870	total: 994ms	remaining: 107ms
903:	learn: 0.3370337	total: 996ms	remaining: 106ms
904:	learn: 0.3370074	total: 997ms	remaining: 105ms
905:	learn: 0.3369451	total: 998ms	remaining: 104ms
906:	learn: 0.3368971	total: 999ms	remaining: 102ms
907:	learn: 

86.04

In [422]:
models=pd.DataFrame({'Model':['Log Reg','Super Vector Machines','Gaussian Naive Bayes','KNN','Perceptron','Linear SVC',
                              'Stochastic Gradient descent','Decision Tree','Random Forest','CatBoost'],
                     'Score':[acc_log,acc_svc,acc_gaussian,acc_knn,acc_per,acc_linearsvc,acc_sgd,acc_dtc,acc_rf,acc_cb]})
models.sort_values(by='Score',ascending=False,ignore_index=True)

,Model,Score
0,Decision Tree,86.04
1,Random Forest,86.04
2,CatBoost,86.04
3,KNN,83.54
4,Super Vector Machines,81.73
5,Linear SVC,80.48
6,Log Reg,80.36
7,Stochastic Gradient descent,80.02
8,Gaussian Naive Bayes,79.57
9,Perceptron,76.62


In [423]:
classifiers=[]
classifiers.append(LogisticRegression())
classifiers.append(SVC())
classifiers.append(GaussianNB())
classifiers.append(KNeighborsClassifier(n_neighbors=5))
classifiers.append(Perceptron())
classifiers.append(LinearSVC())
classifiers.append(SGDClassifier())
classifiers.append(DecisionTreeClassifier())
classifiers.append(RandomForestClassifier(n_estimators=100))
classifiers.append(CatBoostClassifier())
len(classifiers)


10

In [424]:
cv_results=[]
for classifier in classifiers:
    cv_results.append(cross_val_score(classifier,X_train,Y_train,scoring='accuracy',cv=10))

Learning rate set to 0.009326
0:	learn: 0.6865909	total: 1.32ms	remaining: 1.31s
1:	learn: 0.6806446	total: 2.57ms	remaining: 1.28s
2:	learn: 0.6750152	total: 3.69ms	remaining: 1.23s
3:	learn: 0.6691525	total: 4.87ms	remaining: 1.21s
4:	learn: 0.6631288	total: 6.15ms	remaining: 1.22s
5:	learn: 0.6579961	total: 7.28ms	remaining: 1.21s
6:	learn: 0.6528587	total: 8.49ms	remaining: 1.2s
7:	learn: 0.6474434	total: 9.73ms	remaining: 1.21s
8:	learn: 0.6435817	total: 10.7ms	remaining: 1.18s
9:	learn: 0.6389850	total: 11.9ms	remaining: 1.17s
10:	learn: 0.6344522	total: 12.9ms	remaining: 1.16s
11:	learn: 0.6302241	total: 14ms	remaining: 1.15s
12:	learn: 0.6254402	total: 15.2ms	remaining: 1.16s
13:	learn: 0.6204569	total: 16.3ms	remaining: 1.15s
14:	learn: 0.6157905	total: 17.4ms	remaining: 1.14s
15:	learn: 0.6131331	total: 18.1ms	remaining: 1.11s
16:	learn: 0.6089321	total: 19.1ms	remaining: 1.1s
17:	learn: 0.6044886	total: 20.2ms	remaining: 1.1s
18:	learn: 0.5999208	total: 21.2ms	remaining: 1.0

183:	learn: 0.3975967	total: 197ms	remaining: 873ms
184:	learn: 0.3973967	total: 198ms	remaining: 873ms
185:	learn: 0.3971897	total: 199ms	remaining: 873ms
186:	learn: 0.3969069	total: 201ms	remaining: 872ms
187:	learn: 0.3966541	total: 202ms	remaining: 872ms
188:	learn: 0.3964482	total: 203ms	remaining: 871ms
189:	learn: 0.3963400	total: 204ms	remaining: 870ms
190:	learn: 0.3961860	total: 205ms	remaining: 869ms
191:	learn: 0.3958743	total: 206ms	remaining: 868ms
192:	learn: 0.3955051	total: 208ms	remaining: 868ms
193:	learn: 0.3952756	total: 209ms	remaining: 868ms
194:	learn: 0.3949423	total: 210ms	remaining: 868ms
195:	learn: 0.3945426	total: 211ms	remaining: 867ms
196:	learn: 0.3944611	total: 213ms	remaining: 867ms
197:	learn: 0.3942608	total: 214ms	remaining: 868ms
198:	learn: 0.3940232	total: 216ms	remaining: 869ms
199:	learn: 0.3938404	total: 217ms	remaining: 869ms
200:	learn: 0.3935478	total: 219ms	remaining: 869ms
201:	learn: 0.3934341	total: 220ms	remaining: 870ms
202:	learn: 

354:	learn: 0.3687752	total: 397ms	remaining: 721ms
355:	learn: 0.3686473	total: 399ms	remaining: 721ms
356:	learn: 0.3686167	total: 400ms	remaining: 720ms
357:	learn: 0.3684770	total: 401ms	remaining: 719ms
358:	learn: 0.3683633	total: 402ms	remaining: 718ms
359:	learn: 0.3682812	total: 403ms	remaining: 717ms
360:	learn: 0.3681964	total: 405ms	remaining: 716ms
361:	learn: 0.3681059	total: 406ms	remaining: 715ms
362:	learn: 0.3680969	total: 407ms	remaining: 714ms
363:	learn: 0.3680507	total: 408ms	remaining: 713ms
364:	learn: 0.3680074	total: 409ms	remaining: 712ms
365:	learn: 0.3679404	total: 410ms	remaining: 711ms
366:	learn: 0.3677754	total: 412ms	remaining: 710ms
367:	learn: 0.3676921	total: 413ms	remaining: 709ms
368:	learn: 0.3676095	total: 414ms	remaining: 708ms
369:	learn: 0.3675308	total: 415ms	remaining: 707ms
370:	learn: 0.3674818	total: 417ms	remaining: 707ms
371:	learn: 0.3672843	total: 418ms	remaining: 706ms
372:	learn: 0.3671499	total: 419ms	remaining: 705ms
373:	learn: 

546:	learn: 0.3541044	total: 597ms	remaining: 495ms
547:	learn: 0.3540992	total: 599ms	remaining: 494ms
548:	learn: 0.3540527	total: 600ms	remaining: 493ms
549:	learn: 0.3539357	total: 601ms	remaining: 492ms
550:	learn: 0.3539159	total: 602ms	remaining: 491ms
551:	learn: 0.3538248	total: 604ms	remaining: 490ms
552:	learn: 0.3537947	total: 605ms	remaining: 489ms
553:	learn: 0.3537675	total: 606ms	remaining: 488ms
554:	learn: 0.3536972	total: 608ms	remaining: 487ms
555:	learn: 0.3536577	total: 609ms	remaining: 486ms
556:	learn: 0.3536075	total: 610ms	remaining: 485ms
557:	learn: 0.3535994	total: 611ms	remaining: 484ms
558:	learn: 0.3534579	total: 613ms	remaining: 484ms
559:	learn: 0.3534265	total: 614ms	remaining: 483ms
560:	learn: 0.3533269	total: 615ms	remaining: 482ms
561:	learn: 0.3532713	total: 617ms	remaining: 481ms
562:	learn: 0.3531490	total: 618ms	remaining: 479ms
563:	learn: 0.3531002	total: 619ms	remaining: 478ms
564:	learn: 0.3530559	total: 620ms	remaining: 477ms
565:	learn: 

721:	learn: 0.3440401	total: 788ms	remaining: 304ms
722:	learn: 0.3439970	total: 790ms	remaining: 303ms
723:	learn: 0.3439548	total: 791ms	remaining: 302ms
724:	learn: 0.3439064	total: 792ms	remaining: 300ms
725:	learn: 0.3438524	total: 793ms	remaining: 299ms
726:	learn: 0.3438254	total: 795ms	remaining: 298ms
727:	learn: 0.3437490	total: 796ms	remaining: 297ms
728:	learn: 0.3436976	total: 798ms	remaining: 296ms
729:	learn: 0.3436193	total: 799ms	remaining: 296ms
730:	learn: 0.3435706	total: 801ms	remaining: 295ms
731:	learn: 0.3435205	total: 802ms	remaining: 294ms
732:	learn: 0.3434883	total: 804ms	remaining: 293ms
733:	learn: 0.3434208	total: 805ms	remaining: 292ms
734:	learn: 0.3433688	total: 806ms	remaining: 291ms
735:	learn: 0.3433165	total: 807ms	remaining: 289ms
736:	learn: 0.3432115	total: 808ms	remaining: 288ms
737:	learn: 0.3431486	total: 809ms	remaining: 287ms
738:	learn: 0.3431065	total: 811ms	remaining: 286ms
739:	learn: 0.3430582	total: 812ms	remaining: 285ms
740:	learn: 

...

184:	learn: 0.4020799	total: 194ms	remaining: 856ms
185:	learn: 0.4016734	total: 196ms	remaining: 856ms
186:	learn: 0.4013235	total: 197ms	remaining: 856ms
187:	learn: 0.4008985	total: 198ms	remaining: 856ms
188:	learn: 0.4006753	total: 199ms	remaining: 856ms
189:	learn: 0.4004887	total: 201ms	remaining: 855ms
190:	learn: 0.4003616	total: 202ms	remaining: 854ms
191:	learn: 0.4002089	total: 203ms	remaining: 853ms
192:	learn: 0.3998911	total: 204ms	remaining: 852ms
193:	learn: 0.3994773	total: 205ms	remaining: 852ms
194:	learn: 0.3992584	total: 206ms	remaining: 851ms
195:	learn: 0.3989781	total: 207ms	remaining: 850ms
196:	learn: 0.3986085	total: 208ms	remaining: 850ms
197:	learn: 0.3984603	total: 210ms	remaining: 849ms
198:	learn: 0.3982557	total: 211ms	remaining: 851ms
199:	learn: 0.3980126	total: 213ms	remaining: 852ms
200:	learn: 0.3976250	total: 214ms	remaining: 852ms
201:	learn: 0.3974089	total: 215ms	remaining: 851ms
202:	learn: 0.3970353	total: 216ms	remaining: 850ms
203:	learn: 

370:	learn: 0.3716474	total: 398ms	remaining: 674ms
371:	learn: 0.3715993	total: 399ms	remaining: 673ms
372:	learn: 0.3714473	total: 400ms	remaining: 673ms
373:	learn: 0.3713419	total: 401ms	remaining: 672ms
374:	learn: 0.3711635	total: 403ms	remaining: 671ms
375:	learn: 0.3710603	total: 404ms	remaining: 670ms
376:	learn: 0.3709427	total: 405ms	remaining: 669ms
377:	learn: 0.3708220	total: 406ms	remaining: 668ms
378:	learn: 0.3708106	total: 407ms	remaining: 667ms
379:	learn: 0.3707263	total: 408ms	remaining: 666ms
380:	learn: 0.3705926	total: 410ms	remaining: 666ms
381:	learn: 0.3705596	total: 411ms	remaining: 665ms
382:	learn: 0.3703712	total: 413ms	remaining: 665ms
383:	learn: 0.3703613	total: 413ms	remaining: 663ms
384:	learn: 0.3702949	total: 415ms	remaining: 662ms
385:	learn: 0.3701683	total: 416ms	remaining: 661ms
386:	learn: 0.3700383	total: 417ms	remaining: 661ms
387:	learn: 0.3699765	total: 418ms	remaining: 660ms
388:	learn: 0.3699045	total: 420ms	remaining: 659ms
389:	learn: 

542:	learn: 0.3583744	total: 587ms	remaining: 494ms
543:	learn: 0.3582844	total: 589ms	remaining: 494ms
544:	learn: 0.3582385	total: 590ms	remaining: 493ms
545:	learn: 0.3581787	total: 591ms	remaining: 492ms
546:	learn: 0.3581092	total: 593ms	remaining: 491ms
547:	learn: 0.3580177	total: 594ms	remaining: 490ms
548:	learn: 0.3579314	total: 595ms	remaining: 489ms
549:	learn: 0.3578871	total: 596ms	remaining: 488ms
550:	learn: 0.3578289	total: 597ms	remaining: 487ms
551:	learn: 0.3577395	total: 599ms	remaining: 486ms
552:	learn: 0.3576473	total: 600ms	remaining: 485ms
553:	learn: 0.3575597	total: 601ms	remaining: 484ms
554:	learn: 0.3574991	total: 602ms	remaining: 483ms
555:	learn: 0.3574834	total: 603ms	remaining: 482ms
556:	learn: 0.3574276	total: 604ms	remaining: 481ms
557:	learn: 0.3572536	total: 606ms	remaining: 480ms
558:	learn: 0.3571646	total: 607ms	remaining: 479ms
559:	learn: 0.3570765	total: 608ms	remaining: 478ms
560:	learn: 0.3570640	total: 609ms	remaining: 476ms
561:	learn: 

733:	learn: 0.3477705	total: 786ms	remaining: 285ms
734:	learn: 0.3477110	total: 788ms	remaining: 284ms
735:	learn: 0.3476962	total: 789ms	remaining: 283ms
736:	learn: 0.3476528	total: 791ms	remaining: 282ms
737:	learn: 0.3476068	total: 792ms	remaining: 281ms
738:	learn: 0.3475769	total: 793ms	remaining: 280ms
739:	learn: 0.3475141	total: 795ms	remaining: 279ms
740:	learn: 0.3474355	total: 796ms	remaining: 278ms
741:	learn: 0.3473796	total: 798ms	remaining: 277ms
742:	learn: 0.3473257	total: 799ms	remaining: 276ms
743:	learn: 0.3473111	total: 800ms	remaining: 275ms
744:	learn: 0.3472750	total: 801ms	remaining: 274ms
745:	learn: 0.3472347	total: 803ms	remaining: 273ms
746:	learn: 0.3472067	total: 804ms	remaining: 272ms
747:	learn: 0.3471537	total: 805ms	remaining: 271ms
748:	learn: 0.3471358	total: 806ms	remaining: 270ms
749:	learn: 0.3471011	total: 807ms	remaining: 269ms
750:	learn: 0.3470271	total: 808ms	remaining: 268ms
751:	learn: 0.3469960	total: 809ms	remaining: 267ms
752:	learn: 

902:	learn: 0.3391397	total: 979ms	remaining: 105ms
903:	learn: 0.3390903	total: 980ms	remaining: 104ms
904:	learn: 0.3390384	total: 981ms	remaining: 103ms
905:	learn: 0.3390265	total: 982ms	remaining: 102ms
906:	learn: 0.3389735	total: 984ms	remaining: 101ms
907:	learn: 0.3389454	total: 985ms	remaining: 99.8ms
908:	learn: 0.3388730	total: 986ms	remaining: 98.7ms
909:	learn: 0.3388238	total: 987ms	remaining: 97.6ms
910:	learn: 0.3387800	total: 989ms	remaining: 96.6ms
911:	learn: 0.3387065	total: 990ms	remaining: 95.5ms
912:	learn: 0.3386859	total: 991ms	remaining: 94.4ms
913:	learn: 0.3386351	total: 992ms	remaining: 93.4ms
914:	learn: 0.3385924	total: 994ms	remaining: 92.3ms
915:	learn: 0.3385198	total: 995ms	remaining: 91.2ms
916:	learn: 0.3384662	total: 996ms	remaining: 90.1ms
917:	learn: 0.3384116	total: 997ms	remaining: 89.1ms
918:	learn: 0.3384034	total: 998ms	remaining: 88ms
919:	learn: 0.3384028	total: 999ms	remaining: 86.9ms
920:	learn: 0.3383496	total: 1s	remaining: 85.8ms
921

In [425]:
cv_mean=[]
cv_std=[]
for cv_result in cv_results:
    cv_mean.append(cv_result.mean())
    cv_std.append(cv_result.std())

In [426]:
cv_res=pd.DataFrame({'Cross Validation Mean':cv_mean,'Cross Validation Std':cv_std,'Algorithm':['Log Reg',
                            'Super Vector Machines','Gaussian Naive Bayes','KNN','Perceptron','Linear SVC',
                              'Stochastic Gradient descent','Decision Tree','Random Forest','CatBoost']})
cv_res.sort_values(by='Cross Validation Mean', ascending=False,ignore_index=True)

,Cross Validation Mean,Cross Validation Std,Algorithm
0,0.804763,0.033268,CatBoost
1,0.801379,0.024871,KNN
2,0.801366,0.036706,Log Reg
3,0.799119,0.051465,Super Vector Machines
4,0.799106,0.032058,Decision Tree
5,0.793424,0.035446,Linear SVC
6,0.792314,0.035002,Random Forest
7,0.787768,0.042377,Gaussian Naive Bayes
8,0.762704,0.048304,Stochastic Gradient descent
9,0.745761,0.067764,Perceptron


In [436]:
param_grid={'max_depth': [4,5,6,7,8,9,10],'n_estimators':[100, 200, 300]}
grid=GridSearchCV(CatBoostClassifier(),param_grid,refit=True,verbose=3)
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 21 candidates, totalling 105 fits
Learning rate set to 0.073257
0:	learn: 0.6660776	total: 542us	remaining: 53.7ms
1:	learn: 0.6417661	total: 977us	remaining: 47.9ms
2:	learn: 0.6216958	total: 1.36ms	remaining: 44ms
3:	learn: 0.6038268	total: 1.69ms	remaining: 40.5ms
4:	learn: 0.5876317	total: 2.03ms	remaining: 38.6ms
5:	learn: 0.5720012	total: 2.36ms	remaining: 37ms
6:	learn: 0.5568855	total: 2.66ms	remaining: 35.4ms
7:	learn: 0.5441885	total: 2.92ms	remaining: 33.6ms
8:	learn: 0.5337085	total: 3.28ms	remaining: 33.2ms
9:	learn: 0.5227380	total: 3.61ms	remaining: 32.5ms
10:	learn: 0.5120455	total: 3.98ms	remaining: 32.2ms
11:	learn: 0.5035634	total: 4.31ms	remaining: 31.6ms
12:	learn: 0.4954976	total: 4.62ms	remaining: 30.9ms
13:	learn: 0.4886880	total: 4.91ms	remaining: 30.2ms
14:	learn: 0.4810749	total: 5.22ms	remaining: 29.6ms
15:	learn: 0.4747083	total: 5.52ms	remaining: 29ms
16:	learn: 0.4692991	total: 5.82ms	remaining: 28.4ms
17:	learn: 0.4648034	tota

45:	learn: 0.4166585	total: 14.3ms	remaining: 16.8ms
46:	learn: 0.4160113	total: 14.8ms	remaining: 16.7ms
47:	learn: 0.4151317	total: 15.2ms	remaining: 16.4ms
48:	learn: 0.4144041	total: 15.4ms	remaining: 16.1ms
49:	learn: 0.4138461	total: 15.7ms	remaining: 15.7ms
50:	learn: 0.4135534	total: 16.1ms	remaining: 15.4ms
51:	learn: 0.4130923	total: 16.4ms	remaining: 15.2ms
52:	learn: 0.4127345	total: 16.8ms	remaining: 14.9ms
53:	learn: 0.4121434	total: 17.1ms	remaining: 14.5ms
54:	learn: 0.4111978	total: 17.4ms	remaining: 14.2ms
55:	learn: 0.4108759	total: 17.7ms	remaining: 13.9ms
56:	learn: 0.4104067	total: 18ms	remaining: 13.5ms
57:	learn: 0.4101523	total: 18.2ms	remaining: 13.2ms
58:	learn: 0.4096799	total: 18.6ms	remaining: 12.9ms
59:	learn: 0.4088332	total: 18.9ms	remaining: 12.6ms
60:	learn: 0.4083476	total: 19.3ms	remaining: 12.3ms
61:	learn: 0.4075439	total: 19.6ms	remaining: 12ms
62:	learn: 0.4070569	total: 19.9ms	remaining: 11.7ms
63:	learn: 0.4066430	total: 20.2ms	remaining: 11.3

106:	learn: 0.3821494	total: 99.9ms	remaining: 86.8ms
107:	learn: 0.3819950	total: 101ms	remaining: 86.1ms
108:	learn: 0.3818697	total: 102ms	remaining: 85.1ms
109:	learn: 0.3816499	total: 103ms	remaining: 84.3ms
110:	learn: 0.3812405	total: 104ms	remaining: 83.5ms
111:	learn: 0.3809983	total: 105ms	remaining: 82.5ms
112:	learn: 0.3806805	total: 106ms	remaining: 81.6ms
113:	learn: 0.3804257	total: 107ms	remaining: 80.6ms
114:	learn: 0.3799886	total: 108ms	remaining: 79.7ms
115:	learn: 0.3798138	total: 109ms	remaining: 78.7ms
116:	learn: 0.3795443	total: 110ms	remaining: 77.8ms
117:	learn: 0.3794622	total: 111ms	remaining: 76.9ms
118:	learn: 0.3793057	total: 112ms	remaining: 76ms
119:	learn: 0.3790861	total: 113ms	remaining: 75.1ms
120:	learn: 0.3787412	total: 114ms	remaining: 74.2ms
121:	learn: 0.3786429	total: 115ms	remaining: 73.2ms
122:	learn: 0.3785028	total: 115ms	remaining: 72.3ms
123:	learn: 0.3782411	total: 116ms	remaining: 71.4ms
124:	learn: 0.3779109	total: 117ms	remaining: 7

96:	learn: 0.3711734	total: 92.5ms	remaining: 98.2ms
97:	learn: 0.3708259	total: 93.8ms	remaining: 97.6ms
98:	learn: 0.3707509	total: 94.9ms	remaining: 96.8ms
99:	learn: 0.3704878	total: 96ms	remaining: 96ms
100:	learn: 0.3701657	total: 97ms	remaining: 95.1ms
101:	learn: 0.3699880	total: 98ms	remaining: 94.1ms
102:	learn: 0.3697355	total: 99ms	remaining: 93.2ms
103:	learn: 0.3688595	total: 100ms	remaining: 92.5ms
104:	learn: 0.3685563	total: 101ms	remaining: 91.6ms
105:	learn: 0.3683282	total: 103ms	remaining: 90.9ms
106:	learn: 0.3680437	total: 103ms	remaining: 89.9ms
107:	learn: 0.3677552	total: 105ms	remaining: 89.2ms
108:	learn: 0.3674008	total: 106ms	remaining: 88.4ms
109:	learn: 0.3668449	total: 107ms	remaining: 87.6ms
110:	learn: 0.3664303	total: 108ms	remaining: 86.7ms
111:	learn: 0.3663230	total: 109ms	remaining: 85.6ms
112:	learn: 0.3659369	total: 110ms	remaining: 84.6ms
113:	learn: 0.3656275	total: 111ms	remaining: 83.6ms
114:	learn: 0.3650812	total: 112ms	remaining: 82.5ms


64:	learn: 0.4032381	total: 73.4ms	remaining: 152ms
65:	learn: 0.4026364	total: 74.6ms	remaining: 151ms
66:	learn: 0.4019280	total: 76ms	remaining: 151ms
67:	learn: 0.4016120	total: 77.3ms	remaining: 150ms
68:	learn: 0.4012697	total: 78.6ms	remaining: 149ms
69:	learn: 0.4007448	total: 79.7ms	remaining: 148ms
70:	learn: 0.4002079	total: 80.9ms	remaining: 147ms
71:	learn: 0.3993964	total: 82.2ms	remaining: 146ms
72:	learn: 0.3990250	total: 83.3ms	remaining: 145ms
73:	learn: 0.3987137	total: 84.5ms	remaining: 144ms
74:	learn: 0.3983630	total: 85.6ms	remaining: 143ms
75:	learn: 0.3979475	total: 86.9ms	remaining: 142ms
76:	learn: 0.3978223	total: 88ms	remaining: 141ms
77:	learn: 0.3972772	total: 89ms	remaining: 139ms
78:	learn: 0.3969756	total: 90ms	remaining: 138ms
79:	learn: 0.3964997	total: 91ms	remaining: 137ms
80:	learn: 0.3958318	total: 92ms	remaining: 135ms
81:	learn: 0.3953982	total: 93.3ms	remaining: 134ms
82:	learn: 0.3949799	total: 94.5ms	remaining: 133ms
83:	learn: 0.3940079	tot

...

224:	learn: 0.3273868	total: 381ms	remaining: 127ms
225:	learn: 0.3273838	total: 383ms	remaining: 125ms
226:	learn: 0.3272547	total: 386ms	remaining: 124ms
227:	learn: 0.3271312	total: 388ms	remaining: 123ms
228:	learn: 0.3271116	total: 389ms	remaining: 121ms
229:	learn: 0.3268909	total: 392ms	remaining: 119ms
230:	learn: 0.3267531	total: 394ms	remaining: 118ms
231:	learn: 0.3266651	total: 395ms	remaining: 116ms
232:	learn: 0.3265621	total: 397ms	remaining: 114ms
233:	learn: 0.3262629	total: 399ms	remaining: 113ms
234:	learn: 0.3261794	total: 402ms	remaining: 111ms
235:	learn: 0.3260642	total: 404ms	remaining: 110ms
236:	learn: 0.3260617	total: 405ms	remaining: 108ms
237:	learn: 0.3260621	total: 406ms	remaining: 106ms
238:	learn: 0.3258390	total: 408ms	remaining: 104ms
239:	learn: 0.3256314	total: 411ms	remaining: 103ms
240:	learn: 0.3255668	total: 412ms	remaining: 101ms
241:	learn: 0.3254314	total: 414ms	remaining: 99.3ms
242:	learn: 0.3253762	total: 416ms	remaining: 97.5ms
243:	learn

162:	learn: 0.3334591	total: 243ms	remaining: 205ms
163:	learn: 0.3334473	total: 245ms	remaining: 203ms
164:	learn: 0.3334418	total: 246ms	remaining: 201ms
165:	learn: 0.3334249	total: 247ms	remaining: 199ms
166:	learn: 0.3331327	total: 249ms	remaining: 198ms
167:	learn: 0.3326919	total: 252ms	remaining: 198ms
168:	learn: 0.3326699	total: 253ms	remaining: 196ms
169:	learn: 0.3325941	total: 254ms	remaining: 194ms
170:	learn: 0.3325343	total: 256ms	remaining: 193ms
171:	learn: 0.3322174	total: 259ms	remaining: 193ms
172:	learn: 0.3320457	total: 262ms	remaining: 192ms
173:	learn: 0.3318939	total: 264ms	remaining: 191ms
174:	learn: 0.3318805	total: 266ms	remaining: 190ms
175:	learn: 0.3314141	total: 269ms	remaining: 189ms
176:	learn: 0.3313652	total: 270ms	remaining: 188ms
177:	learn: 0.3313444	total: 272ms	remaining: 186ms
178:	learn: 0.3309642	total: 274ms	remaining: 185ms
179:	learn: 0.3308413	total: 276ms	remaining: 184ms
180:	learn: 0.3304929	total: 278ms	remaining: 183ms
181:	learn: 

94:	learn: 0.3655433	total: 134ms	remaining: 289ms
95:	learn: 0.3648280	total: 136ms	remaining: 290ms
96:	learn: 0.3643485	total: 138ms	remaining: 289ms
97:	learn: 0.3635747	total: 141ms	remaining: 290ms
98:	learn: 0.3626869	total: 143ms	remaining: 290ms
99:	learn: 0.3619678	total: 145ms	remaining: 290ms
100:	learn: 0.3614912	total: 147ms	remaining: 290ms
101:	learn: 0.3608879	total: 150ms	remaining: 291ms
102:	learn: 0.3603520	total: 152ms	remaining: 291ms
103:	learn: 0.3599494	total: 156ms	remaining: 294ms
104:	learn: 0.3599253	total: 157ms	remaining: 292ms
105:	learn: 0.3597563	total: 159ms	remaining: 290ms
106:	learn: 0.3596408	total: 160ms	remaining: 289ms
107:	learn: 0.3596116	total: 161ms	remaining: 286ms
108:	learn: 0.3591623	total: 163ms	remaining: 286ms
109:	learn: 0.3591236	total: 164ms	remaining: 283ms
110:	learn: 0.3584839	total: 166ms	remaining: 283ms
111:	learn: 0.3583451	total: 167ms	remaining: 281ms
112:	learn: 0.3580852	total: 169ms	remaining: 280ms
113:	learn: 0.3580

0:	learn: 0.6615720	total: 583us	remaining: 57.8ms
1:	learn: 0.6352175	total: 1.19ms	remaining: 58.1ms
2:	learn: 0.6145558	total: 1.59ms	remaining: 51.4ms
3:	learn: 0.5956235	total: 1.96ms	remaining: 47.1ms
4:	learn: 0.5792181	total: 2.33ms	remaining: 44.4ms
5:	learn: 0.5632801	total: 2.67ms	remaining: 41.8ms
6:	learn: 0.5479650	total: 3.03ms	remaining: 40.2ms
7:	learn: 0.5350293	total: 3.37ms	remaining: 38.7ms
8:	learn: 0.5228519	total: 3.77ms	remaining: 38.1ms
9:	learn: 0.5130273	total: 4.09ms	remaining: 36.8ms
10:	learn: 0.5041313	total: 4.44ms	remaining: 36ms
11:	learn: 0.4951729	total: 4.83ms	remaining: 35.4ms
12:	learn: 0.4892015	total: 5.21ms	remaining: 34.9ms
13:	learn: 0.4827017	total: 5.55ms	remaining: 34.1ms
14:	learn: 0.4756080	total: 5.99ms	remaining: 33.9ms
15:	learn: 0.4702092	total: 6.38ms	remaining: 33.5ms
16:	learn: 0.4644310	total: 6.78ms	remaining: 33.1ms
17:	learn: 0.4597776	total: 7.16ms	remaining: 32.6ms
18:	learn: 0.4554186	total: 7.54ms	remaining: 32.2ms
19:	le

GridSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x000002A2E8FF8690>,
             param_grid={'max_depth': [4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [100, 200, 300]},
             verbose=3)

In [437]:
print("Best Parameters:",grid.best_params_)
print("Best Estimator:",grid.best_estimator_)

Best Parameters: {'max_depth': 4, 'n_estimators': 100}
Best Estimator: <catboost.core.CatBoostClassifier object at 0x000002A2E7D7C150>


In [438]:
cbc=CatBoostClassifier(max_depth=4,n_estimators=100)
cbc.fit(X_train,Y_train)
Y_pred=cbc.predict(X_test)
acc_cbc=round(cbc.score(X_train,Y_train)*100,2)
acc_cbc

Learning rate set to 0.080619
0:	learn: 0.6615720	total: 452us	remaining: 44.8ms
1:	learn: 0.6352175	total: 791us	remaining: 38.8ms
2:	learn: 0.6145558	total: 1.11ms	remaining: 35.8ms
3:	learn: 0.5956235	total: 1.43ms	remaining: 34.3ms
4:	learn: 0.5792181	total: 1.89ms	remaining: 36ms
5:	learn: 0.5632801	total: 2.26ms	remaining: 35.4ms
6:	learn: 0.5479650	total: 2.6ms	remaining: 34.5ms
7:	learn: 0.5350293	total: 2.92ms	remaining: 33.6ms
8:	learn: 0.5228519	total: 3.29ms	remaining: 33.3ms
9:	learn: 0.5130273	total: 3.63ms	remaining: 32.7ms
10:	learn: 0.5041313	total: 3.97ms	remaining: 32.2ms
11:	learn: 0.4951729	total: 4.29ms	remaining: 31.5ms
12:	learn: 0.4892015	total: 4.66ms	remaining: 31.2ms
13:	learn: 0.4827017	total: 4.99ms	remaining: 30.7ms
14:	learn: 0.4756080	total: 5.34ms	remaining: 30.3ms
15:	learn: 0.4702092	total: 5.67ms	remaining: 29.8ms
16:	learn: 0.4644310	total: 6ms	remaining: 29.3ms
17:	learn: 0.4597776	total: 6.33ms	remaining: 28.9ms
18:	learn: 0.4554186	total: 6.66ms

83.54

In [439]:
cross_val_score(cbc,X_train,Y_train,scoring='accuracy',cv=10).mean()

Learning rate set to 0.077035
0:	learn: 0.6631840	total: 930us	remaining: 92.1ms
1:	learn: 0.6379715	total: 1.74ms	remaining: 85.2ms
2:	learn: 0.6154845	total: 2.37ms	remaining: 76.7ms
3:	learn: 0.5975068	total: 3.15ms	remaining: 75.7ms
4:	learn: 0.5809656	total: 3.93ms	remaining: 74.6ms
5:	learn: 0.5643694	total: 4.98ms	remaining: 78.1ms
6:	learn: 0.5497139	total: 5.75ms	remaining: 76.5ms
7:	learn: 0.5374429	total: 6.41ms	remaining: 73.8ms
8:	learn: 0.5269866	total: 7.16ms	remaining: 72.4ms
9:	learn: 0.5170188	total: 7.82ms	remaining: 70.4ms
10:	learn: 0.5067049	total: 8.55ms	remaining: 69.2ms
11:	learn: 0.4984285	total: 9.27ms	remaining: 68ms
12:	learn: 0.4907411	total: 9.96ms	remaining: 66.6ms
13:	learn: 0.4853428	total: 10.6ms	remaining: 64.9ms
14:	learn: 0.4777830	total: 11.3ms	remaining: 64ms
15:	learn: 0.4718549	total: 12.1ms	remaining: 63.7ms
16:	learn: 0.4666760	total: 12.8ms	remaining: 62.7ms
17:	learn: 0.4625303	total: 13.6ms	remaining: 61.9ms
18:	learn: 0.4598686	total: 14.

64:	learn: 0.4081534	total: 21.6ms	remaining: 11.6ms
65:	learn: 0.4075855	total: 22.2ms	remaining: 11.5ms
66:	learn: 0.4071094	total: 22.6ms	remaining: 11.1ms
67:	learn: 0.4067224	total: 23ms	remaining: 10.8ms
68:	learn: 0.4063915	total: 23.3ms	remaining: 10.5ms
69:	learn: 0.4057232	total: 23.7ms	remaining: 10.1ms
70:	learn: 0.4051837	total: 24ms	remaining: 9.79ms
71:	learn: 0.4046911	total: 24.3ms	remaining: 9.45ms
72:	learn: 0.4042221	total: 24.7ms	remaining: 9.12ms
73:	learn: 0.4040344	total: 25ms	remaining: 8.78ms
74:	learn: 0.4038045	total: 25.3ms	remaining: 8.43ms
75:	learn: 0.4030255	total: 25.6ms	remaining: 8.08ms
76:	learn: 0.4028065	total: 26.1ms	remaining: 7.78ms
77:	learn: 0.4023149	total: 26.4ms	remaining: 7.46ms
78:	learn: 0.4014919	total: 26.8ms	remaining: 7.12ms
79:	learn: 0.4011500	total: 27.1ms	remaining: 6.77ms
80:	learn: 0.4008886	total: 27.4ms	remaining: 6.43ms
81:	learn: 0.4001389	total: 27.8ms	remaining: 6.1ms
82:	learn: 0.3995035	total: 28.2ms	remaining: 5.77ms


Learning rate set to 0.077077
0:	learn: 0.6617071	total: 435us	remaining: 43.1ms
1:	learn: 0.6366840	total: 753us	remaining: 36.9ms
2:	learn: 0.6167598	total: 1.06ms	remaining: 34.3ms
3:	learn: 0.5976181	total: 1.38ms	remaining: 33ms
4:	learn: 0.5807125	total: 1.68ms	remaining: 32ms
5:	learn: 0.5652924	total: 1.99ms	remaining: 31.1ms
6:	learn: 0.5502108	total: 2.28ms	remaining: 30.4ms
7:	learn: 0.5377165	total: 2.52ms	remaining: 29ms
8:	learn: 0.5283685	total: 2.82ms	remaining: 28.5ms
9:	learn: 0.5180056	total: 3.12ms	remaining: 28ms
10:	learn: 0.5071842	total: 3.42ms	remaining: 27.7ms
11:	learn: 0.4989172	total: 3.72ms	remaining: 27.3ms
12:	learn: 0.4908714	total: 4.04ms	remaining: 27ms
13:	learn: 0.4842661	total: 4.34ms	remaining: 26.7ms
14:	learn: 0.4770113	total: 4.63ms	remaining: 26.2ms
15:	learn: 0.4709648	total: 4.92ms	remaining: 25.8ms
16:	learn: 0.4658901	total: 5.26ms	remaining: 25.7ms
17:	learn: 0.4616681	total: 5.56ms	remaining: 25.3ms
18:	learn: 0.4571988	total: 5.87ms	rem

0.8115806945863125

In [440]:
Y_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [441]:
len(Y_pred)

418

In [442]:
ss.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [443]:
ss.shape

(418, 2)

In [444]:
final_prediction=pd.DataFrame({'Passenger ID': test.PassengerId, 'Survived': Y_pred})
final_prediction

,Passenger ID,Survived
881,892,0
882,893,0
883,894,0
884,895,0
885,896,0
...,...,...
1294,1305,0
1295,1306,1
1296,1307,0
1297,1308,0
